In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import json
import baseline
from baseline import *
from decoder import *
from alphabet import *
from train import *
import torch
from matplotlib import pyplot as plt

torch.__version__

'1.4.0'

In [11]:
LANGUAGE = 'Python'

def filter_ascii(strings):
    'Returns only the strings that can be encoded in ASCII.'
    l = []
    for s in strings:
        try:
            s.encode('ascii')
            if len(s) <= 80:
                l.append(s)
        except UnicodeEncodeError:
            pass
        
    return l

with open('dataset/medium.json') as f:
    multilang_dataset = json.load(f)
    dataset = multilang_dataset[LANGUAGE]
    
    dataset['train'] = filter_ascii(dataset['train'])
    dataset['dev'] = filter_ascii(dataset['dev'])
    dataset['test'] = filter_ascii(dataset['test'])
    
    tiny_dataset = {
        'train': dataset['train'][:50],
        'dev': dataset['train'][:50],
        'test': dataset['train'][:50],
    }
    
    print('{} training examples, {} validation examples, {} test exampless'.format(
        len(dataset['train']), 
        len(dataset['dev']),
        len(dataset['test'])))

76738 training examples, 9590 validation examples, 9616 test exampless


In [3]:
def generate_dumb_dataset():
    'Returns a dataset where all examples are the same string, which consists of 10 times the same letter.'

    SIZE = 200
    l = []

    for i in range(SIZE):
        l.append(random.choice('abcdefghijklmnopqrstuvwxyz') * random.choice([5, 10]))
        
    return {'train': l, 'dev': l, 'test': l}

dumb_dataset = generate_dumb_dataset()

In [43]:
device = torch.device(0) if torch.cuda.is_available() else torch.device('cpu')
print(device)

alphabet = AsciiOneHotEncoding(device)
encoder = baseline.UniformEncoder(0.9)
decoder = AutoCompleteDecoderModel(alphabet, hidden_size=256)

cuda:0


In [ ]:
parameters = {
    'learning_rate': 5e-4,
    'verbose': True,
    'batch_size': 64,
    'init_scale': 0.001,
    'epochs': 50,
}

train_loss_history = train(encoder, decoder, dataset, parameters, device)
plt.plot(train_loss_history)

Epoch 0 iteration 0: loss = 4.852, tp = 286.44 lines/s
Epoch 0 iteration 100: loss = 3.568, tp = 388.14 lines/s
Epoch 0 iteration 200: loss = 3.600, tp = 386.94 lines/s


In [42]:
train_loss_history

[4.712904930114746,
 3.890644073486328,
 3.70444393157959,
 4.679772853851318,
 1.9984501600265503,
 3.2455244064331055,
 5.587140083312988,
 3.8608806133270264,
 4.561099529266357,
 4.606544494628906,
 5.184301853179932,
 4.412602424621582,
 4.919872760772705,
 3.2686870098114014,
 4.634662628173828,
 4.948631286621094,
 3.3313193321228027,
 3.993622064590454,
 4.51015043258667,
 3.075333833694458,
 4.173201084136963,
 4.405919075012207,
 4.000620365142822,
 3.4062323570251465,
 3.5477828979492188,
 2.301224708557129,
 6.405610084533691,
 3.2499938011169434,
 1.7814885377883911,
 3.456925868988037,
 2.8369264602661133,
 4.202714443206787,
 3.9620020389556885,
 3.485891103744507,
 3.8492324352264404,
 3.675689458847046,
 2.9940221309661865,
 3.385756254196167,
 4.140872001647949,
 4.094381332397461,
 4.111380100250244,
 2.1043643951416016,
 2.659740686416626,
 2.1758687496185303,
 3.1547441482543945,
 3.077089548110962,
 2.863400936126709,
 2.521085739135742,
 1.7278597354888916,
 1.80

# End-to-end samples

In [40]:
SPLIT = 'train'

import copy

s = random.choice(dumb_dataset[SPLIT])
compressed = encoder.encode(s)
decompressed = decoder([compressed])

print('String:', repr(s))
print('Encoded:', repr(compressed))
print('Decoded:', repr(decompressed[0]))

String: 'ffffffffff'
Encoded: 'ffffffffff'
Decoded: 'ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'
